# Doc Embedding

In [1]:
# obtain the data
!wget http://alt.qcri.org/semeval2017/task1/data/uploads/sts2017.eval.v1.1.zip
!wget http://alt.qcri.org/semeval2017/task1/data/uploads/sts2017.gs.zip

!unzip sts2017.eval.v1.1.zip 
!unzip sts2017.gs.zip



--2023-03-30 07:27:13--  http://alt.qcri.org/semeval2017/task1/data/uploads/sts2017.eval.v1.1.zip
Resolving alt.qcri.org (alt.qcri.org)... 80.76.166.234
Connecting to alt.qcri.org (alt.qcri.org)|80.76.166.234|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://alt.qcri.org/semeval2017/task1/data/uploads/sts2017.eval.v1.1.zip [following]
--2023-03-30 07:27:13--  https://alt.qcri.org/semeval2017/task1/data/uploads/sts2017.eval.v1.1.zip
Connecting to alt.qcri.org (alt.qcri.org)|80.76.166.234|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87902 (86K) [application/zip]
Saving to: ‘sts2017.eval.v1.1.zip’

sts2017.eval.v1.1.z 100%[===================>]  85.84K   426KB/s    in 0.2s    

2023-03-30 07:27:14 (426 KB/s) - ‘sts2017.eval.v1.1.zip’ saved [87902/87902]

URL transformed to HTTPS due to an HSTS policy
--2023-03-30 07:27:15--  https://alt.qcri.org/semeval2017/task1/data/uploads/sts2017.gs.zip
Resolving alt.qcri.org

In [2]:
# load the data

def load_STS_data():
    with open("STS2017.gs/STS.gs.track5.en-en.txt") as f:
        labels = [float(line.strip()) for line in f]
    
    text_a, text_b = [], []
    with open("STS2017.eval.v1.1/STS.input.track5.en-en.txt") as f:
        for line in f:
            line = line.strip().split("\t")
            text_a.append(line[0])
            text_b.append(line[1])
    return text_a, text_b, labels

text_a, text_b, labels = load_STS_data()
text_a[0], text_b[0], labels[0]

('A person is on a baseball team.',
 'A person is playing basketball on a team.',
 2.4)

In [3]:
# some utils
from scipy.stats import spearmanr
def evaluate(predictions, labels):
    print ("spearman's rank correlation", spearmanr(predictions, labels)[0])

import numpy as np
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(a,b):
    return dot(a, b)/(norm(a)*norm(b))



In [ ]:
# Wordcounts baseline
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
vec.fit(text_a + text_b)

# encode documents
text_a_encoded = np.array(vec.transform(text_a).todense())
text_b_encoded = np.array(vec.transform(text_b).todense())

# predict cosine similarities
predictions = [cosine_similarity(a,b) for a,b in zip(text_a_encoded, text_b_encoded)]

# evaluate
evaluate(predictions, labels)

spearman's rank correlation 0.6998056665685976


In [ ]:
# Wordcounts baseline
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
vec.fit(text_a + text_b)

# encode documents
text_a_encoded = np.array(vec.transform(text_a).todense())
text_b_encoded = np.array(vec.transform(text_b).todense())

# predict cosine similarities
predictions = [cosine_similarity(a,b) for a,b in zip(text_a_encoded, text_b_encoded)]

# evaluate
evaluate(predictions, labels)

spearman's rank correlation 0.6998056665685976


In [9]:
##TODO train Doc2Vec on the texts in the dataset
##TODO derive the word vectors for each text in the dataset
##TODO compute cosine similarity between the text pairs and evaluate spearman's rank correlation

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk
from nltk import word_tokenize
nltk.download('punkt')
docs = []

for i in text_a + text_b:
    docs.append(word_tokenize(i))

# we have to encode the unique id for each document
doc_iterator = [TaggedDocument(doc,[i]) for i, doc in enumerate(docs)]
d2v = Doc2Vec(doc_iterator,
              min_count=2,
              window=5,
              vector_size=25,
              sample=1e-4,
              negative=5,
              workers=4,
              max_vocab_size=1000)


text_a_encoded = d2v.dv.vectors[:len(text_a)]
text_b_encoded = d2v.dv.vectors[len(text_a):]

pred = [cosine_similarity(a, b) for a, b in zip(text_a_encoded, text_b_encoded)]

evaluate(pred, labels)

spearman's rank correlation 0.014730844630254609


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
##TODO do the same with embeddings provided by spaCy

import spacy

nlp = spacy.load('en_core_web_sm')

text_a_encoded = [nlp(text).vector for text in text_a]
text_b_encoded = [nlp(text).vector for text in text_b]

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [11]:
predictions = [cosine_similarity(a, b) for a, b in zip(text_a_encoded, text_b_encoded)]
evaluate(predictions, labels)

spearman's rank correlation 0.48796451382976114


In [15]:
! pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 73.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=d71edb92d0a4d2a0a9c9f0f6435cd2a07da2fe393a0cf932d4be47414a51c522
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


In [16]:
##TODO do the same with SBERT embeddings

from sentence_transformers import SentenceTransformer
model = "bert-base-nli-mean-tokens"
embedder = SentenceTransformer(model)
text_a_encoded = embedder.encode(text_a)
text_b_encoded = embedder.encode(text_b)

predictions = [cosine_similarity(a, b) for a,b in zip(text_a_encoded, text_b_encoded)]

evaluate(predictions, labels)

spearman's rank correlation 0.8008164100246977
